In [1]:
import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)


import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()) and "anuja" in os.environ.get('USER'):
    DATA_DIR = 'data/'
    

## EEG data and foof:

In [25]:
## data loading
data = mat73.loadmat('x_source_new.mat')  
foof = pd.read_csv(DATA_DIR+"/home/ubuntu/Martyna/repo/AI4Health/DATAfoof/sensor_intercept_slope.csv")
foof

,0,1,2,3,4,5,6,7,8,9,...,201,202,203,204,205,206,207,208,209,IDs
0,1.610769,1.793923,1.639958,1.803579,1.622763,1.867163,1.596286,1.868653,1.459489,1.880624,...,1.637747,1.472414,1.775967,1.534612,1.803604,1.575800,1.869977,0.986272,1.825774,NDARAA075AMK
1,1.572269,1.762184,1.615888,1.938907,1.515331,1.878222,1.577389,1.888830,1.645799,1.892993,...,1.698141,1.698537,1.854421,1.486611,1.755794,1.517099,1.845983,1.486650,1.888544,NDARAA112DMH
2,1.414507,1.818785,1.504383,1.902537,1.563395,1.936381,1.554663,2.085351,1.566065,2.106747,...,1.847331,1.417519,1.869143,1.323095,1.838333,1.496008,1.895424,1.593155,2.095749,NDARAA117NEJ
3,0.561062,1.068994,0.333554,0.991180,0.555779,1.304169,0.665864,1.491464,0.524828,1.573271,...,1.464311,0.576556,1.265274,0.199181,0.863242,0.246682,1.010393,0.703331,1.724831,NDARAA947ZG5
4,1.262007,1.901401,1.305927,1.924721,1.293914,1.924840,1.226456,1.947274,0.818595,1.905113,...,1.859064,1.158560,1.863190,1.173287,1.773532,1.252168,1.878925,0.918020,1.749441,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,0.648706,1.508826,0.591809,1.640814,0.640782,1.665891,0.455759,1.679019,0.866016,1.786340,...,1.536850,0.528776,1.478037,0.466827,1.767683,0.570274,1.664126,1.351549,1.996940,NDARZN277NR6
2038,1.332278,1.729382,1.387863,1.788208,1.452167,1.841329,1.524229,1.976239,1.480653,2.022584,...,1.876485,1.214977,1.795080,1.456392,1.835807,1.399127,1.795906,1.380795,2.036327,NDARZN578YDP
2039,0.746123,1.324138,0.653335,1.285734,0.564490,1.271931,0.375903,1.245331,0.258508,1.206112,...,1.307891,0.691917,1.310004,0.604437,1.283587,0.564744,1.273098,0.339229,1.050644,NDARZN610GTY
2040,0.976055,1.441830,0.823197,1.407064,0.879368,1.464799,0.902699,1.512205,1.012450,1.505105,...,1.341166,0.598687,1.282644,0.697733,1.421177,0.753861,1.457204,0.781225,1.470061,NDARZN677EYE


In [26]:
# flattening
df2 = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df2 = np.array(df2)
df2 = df2.reshape((2042,68,79)) 
df2.shape

(2042, 68, 79)

In [27]:
df2 = pd.DataFrame(df2.reshape((df2.shape[0], -1)))
df2 = pd.DataFrame(df2)
type(df2)
df2['IDs'] = foof['IDs']
df2

,0,1,2,3,4,5,6,7,8,9,...,5363,5364,5365,5366,5367,5368,5369,5370,5371,IDs
0,2.852107e-12,6.843959e-11,1.334486e-09,2.114389e-08,2.722198e-07,2.847860e-06,2.420928e-05,1.672280e-04,9.386428e-04,4.281104e-03,...,4.083400e-13,1.199041e-14,2.220446e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA075AMK
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.422859e-02,1.792809e-01,2.261597e-01,1.471350e-01,4.932961e-02,8.522938e-03,7.588586e-04,3.481949e-05,8.233298e-07,NDARAA112DMH
2,1.873727e-04,3.700203e-02,1.338341e-01,8.866054e-03,1.075761e-05,2.390714e-10,1.143530e-13,4.469758e-12,1.420740e-10,4.289156e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA117NEJ
3,4.679133e-03,2.858350e-02,1.446466e-01,3.296430e-01,3.146601e-01,1.356668e-01,1.148714e-01,3.878903e-01,4.688989e-01,1.760009e-01,...,4.721801e-02,2.446504e-02,6.717445e-02,1.737527e-01,2.703009e-01,2.480023e-01,1.340995e-01,4.273146e-02,8.024469e-03,NDARAA947ZG5
4,1.426186e-01,1.919957e-01,2.337057e-01,2.572226e-01,2.559831e-01,2.303433e-01,1.874140e-01,1.378767e-01,9.171530e-02,5.516572e-02,...,5.047329e-02,4.878144e-02,5.355981e-02,1.013861e-01,5.092052e-02,3.967086e-02,3.699821e-02,3.428801e-02,3.155638e-02,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,1.641564e-04,5.061825e-03,4.748293e-02,1.355030e-01,1.176366e-01,3.107015e-02,2.513731e-03,1.849108e-04,7.014072e-04,3.183943e-03,...,3.161835e-08,1.106568e-08,3.738061e-09,1.218730e-09,3.834910e-10,1.164628e-10,3.413536e-11,9.656276e-12,2.636114e-12,NDARZN277NR6
2038,1.273725e-07,2.854585e-06,4.301847e-05,4.342317e-04,2.934193e-03,1.327155e-02,4.019215e-02,8.169716e-02,1.133810e-01,1.182013e-01,...,6.180279e-12,1.819545e-12,5.200285e-13,1.443290e-13,3.885781e-14,1.021405e-14,2.553513e-15,6.661338e-16,1.110223e-16,NDARZN578YDP
2039,0.000000e+00,7.771561e-16,3.630429e-14,1.391887e-12,4.209189e-11,1.004152e-09,1.889601e-08,2.804845e-07,3.284092e-06,3.033118e-05,...,1.329739e-01,1.350156e-01,1.346921e-01,1.320204e-01,1.271395e-01,1.202986e-01,1.118360e-01,1.021511e-01,9.167378e-02,NDARZN610GTY
2040,1.363800e-08,1.521235e-07,1.428039e-06,1.127810e-05,7.492867e-05,4.187613e-04,1.968741e-03,7.785975e-03,2.590237e-02,7.248831e-02,...,5.546025e-07,2.166196e-07,8.182735e-08,2.989401e-08,1.056221e-08,3.609193e-09,1.192753e-09,3.812199e-10,1.178382e-10,NDARZN677EYE


## Behavioral Data:


In [28]:
behaviour_data = pd.read_csv('/home/ubuntu/Martyna/repo/AI4Health/AllData.csv')
behaviour_data = behaviour_data.rename(columns = {'EID': 'IDs'}, inplace = False)
behaviour_data.columns

Index(['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_SR_GD', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum'],
      dtype='object')

In [29]:
behaviour_data = behaviour_data[['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum']]
behaviour_data.shape

(2579, 9)

In [30]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(behaviour_data)

Your selected dataframe has 9 columns (with column 'IDs').
There are 8 columns that have missing values.


,Missing Values,% of Total Values
WISC_WMI_Sum,712,27.6
WISC_VCI_Sum,702,27.2
SCARED_P_GD,576,22.3
SRS_SCI_T,448,17.4
SRS_RRB_T,447,17.3
SWAN_IN_Avg,385,14.9
SWAN_HY_Avg,385,14.9
SWAN_Avg,385,14.9


## Merge EEG and behavioral data together:

In [31]:
df = pd.merge(df2, behaviour_data, on='IDs', how='inner')
df.shape

(1465, 5381)

## features and labels preparation:

In [32]:
#removing NaNs
df = df.dropna()
df.shape

(1075, 5381)

In [33]:
X = df.iloc[:, 0:5371].to_numpy()
X.shape

(1075, 5371)

In [34]:
Y = df[[ 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum']]

## TARGETS NORMALISATION
Y = Y.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
Y_scaled = min_max_scaler.fit_transform(Y)
Y = pd.DataFrame(Y_scaled)
print(Y.shape)
Y.mean()

(1075, 8)


0    0.412300
1    0.330517
2    0.618501
3    0.542498
4    0.580500
5    0.260155
6    0.456076
7    0.494200
dtype: float64

## PCA on X dimension



In [45]:
# dimensionality reduction
pca = PCA(.90) # 95% variance retained
pca.fit(X)
# transform data
X_pca = pca.transform(X)
X_pca.shape

X = X_pca
print("X:", X.shape, "Y:", Y.shape)


X: (1075, 100) Y: (1075, 8)


In [46]:
in_dim = X.shape[1]
out_dim = Y.shape[1]
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.15)

## MODELLING PART

### dummy regressors (to obtain the random baseline):

In [47]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.dummy import DummyRegressor



xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.5)
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(xtrain, ytrain)
lm_dummy_median = DummyRegressor(strategy = 'median').fit(xtrain, ytrain)
ypred_dummy_mean = lm_dummy_mean.predict(xtest)
ypred_dummy_median = lm_dummy_median.predict(xtest)


print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(ytest,
																	ypred_dummy_mean)))

print("Median absolute error (dummy): {:.2f}".format(median_absolute_error(ytest,
																	ypred_dummy_median)))

print("r2_score (dummy mean): {:.2f}".format(r2_score(ytest, ypred_dummy_mean)))
print("r2_score (dummy median): {:.2f}".format(r2_score(ytest, ypred_dummy_median)))

print(ytest[:10])
print(Y.mean())

Mean squared error (dummy): 0.04
Median absolute error (dummy): 0.12
r2_score (dummy mean): -0.00
r2_score (dummy median): -0.02
             0         1         2         3         4         5        6  \
697   0.129630  0.020408  0.592592  0.500000  0.546296  0.000000  0.78125   
190   0.240741  0.000000  0.740741  0.518518  0.629630  0.277778  0.31250   
116   0.277778  0.142857  0.666667  0.500000  0.583333  0.222222  0.50000   
48    0.148148  0.142857  0.481482  0.351852  0.416667  0.055556  0.56250   
420   0.722222  0.551020  0.648148  0.592592  0.620370  0.055556  0.03125   
835   0.481481  0.367347  0.611111  0.592592  0.601852  0.111111  0.62500   
1011  0.129630  0.020408  0.277778  0.055556  0.166667  0.000000  0.56250   
850   0.370370  0.224490  0.592592  0.962963  0.777778  0.166667  0.53125   
395   0.296296  0.265306  0.629630  0.370370  0.500000  0.388889  0.50000   
432   0.425926  0.142857  0.759259  0.666667  0.712963  0.055556  0.43750   

             7  
697   

### SVR regressor

In [48]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Create the SVR regressor
svr = SVR(epsilon=0.2)
#Create the Multioutput Regressor
mor = MultiOutputRegressor(svr)
# Train the regressor
mor = mor.fit(xtrain, ytrain)
# Generate predictions for testing data
ypred = mor.predict(xtest)


print("Mean squared error (SVC): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))



Mean squared error (SVC): 0.05


## keras, simple MLP


In [49]:
model = Sequential()
model.add(Dense(100, input_dim=in_dim, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(out_dim))
model.compile(loss="mse", metrics = ['mse'], optimizer="adam")


model.fit(xtrain, ytrain, epochs=100, batch_size=12, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)




Epoch 1/100
36/36 [==============================] - 0s 3ms/step - loss: 0.2923 - mse: 0.2923 - val_loss: 0.1286 - val_mse: 0.1286
Epoch 2/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0904 - mse: 0.0904 - val_loss: 0.0919 - val_mse: 0.0919
Epoch 3/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0656 - mse: 0.0656 - val_loss: 0.0808 - val_mse: 0.0808
Epoch 4/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0546 - mse: 0.0546 - val_loss: 0.0759 - val_mse: 0.0759
Epoch 5/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0468 - mse: 0.0468 - val_loss: 0.0746 - val_mse: 0.0746
Epoch 6/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0417 - mse: 0.0417 - val_loss: 0.0704 - val_mse: 0.0704
Epoch 7/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0374 - mse: 0.0374 - val_loss: 0.0689 - val_mse: 0.0689
Epoch 8/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0340 - m

In [50]:
score = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)

score= np.array(score)

score.mean()

0.0861804

In [51]:
#xtrain = np.asarray(xtrain).astype('float32')
#xtest = np.asarray(xtest).astype('float32')